In [2]:
import tensorflow as tf
# ^^^ pyforest auto-imports - don't write above this line


In [3]:
import yfinance as yf
import sklearn
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers


### Basic TF syntax
 - Similar to Numpy

In [4]:
tf.zeros(10)

<IPython.core.display.Javascript object>

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [5]:
initial_value = tf.random.normal(shape=(2, 2))
a = tf.Variable(initial_value)
b= tf.Variable(initial_value)*a
a,b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
 array([[ 0.3118146 ,  0.00751485],
        [-1.7428792 ,  0.4655131 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[9.7228348e-02, 5.6473044e-05],
        [3.0376277e+00, 2.1670246e-01]], dtype=float32)>)

In [6]:
with tf.GradientTape() as tape:
    c = tf.sqrt(tf.square(a) + tf.square(b)) # tensorflow expressions
    dc_da = tape.gradient(c, a) # take gradient
    print(dc_da)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

tf.Tensor(
[[ 0.9546661   0.99997175]
 [-0.4976647   0.9065834 ]], shape=(2, 2), dtype=float32)


### Activation Layers

Common functions are 
- sigmoid
- relu
- tanh
- softplus
- softmax

### Dense Layers

Each dense layer is the linear functional equation (function need not be linear) s.t.

output= activation(inner_product(input,kernel)+bias)

Where
- Activation is above
- Kernel are weights at each layer
- Bias is some bias when input is 0

### Simple Neural Network

In [7]:
aapl = yf.Ticker('AAPL')
history = aapl.history(period='7d', interval='1m')

history=history.drop(columns=['Dividends','Stock Splits'])
history['Next_Open']=history['Open'].shift(-1)
history['Next_Open']=(history['Next_Open']>history['Close']).astype(int)
hisory=history.dropna()
history

,Open,High,Low,Close,Volume,Next_Open
Datetime,,,,,,
2022-06-02 09:30:00-04:00,147.830002,148.160004,147.320007,147.490005,2526172,0
2022-06-02 09:31:00-04:00,147.490005,147.610001,146.860001,147.059998,586575,1
2022-06-02 09:32:00-04:00,147.070007,147.470001,147.070007,147.419998,435226,1
2022-06-02 09:33:00-04:00,147.440002,147.589996,147.149994,147.509995,290213,0
2022-06-02 09:34:00-04:00,147.506104,147.919998,147.350006,147.750000,419969,0
...,...,...,...,...,...,...
2022-06-10 11:28:00-04:00,137.390503,137.410004,137.270004,137.380096,222146,1
2022-06-10 11:29:00-04:00,137.389893,137.460007,137.300003,137.399994,190395,1
2022-06-10 11:30:00-04:00,137.410995,137.520004,137.380005,137.419998,160326,0


### Use Previous OHLCV to predict next open 

In [8]:
features=['Open','High','Low','Close','Volume']

### Separata Data into training/testing

In [9]:
### sklearn method to randomly sample train/test data -> Assuming no time series effect 
X_train, X_test, Y_train, Y_test = train_test_split(history[features], history['Next_Open'], random_state=0, train_size = .75)
X_train

,Open,High,Low,Close,Volume
Datetime,,,,,
2022-06-10 09:39:00-04:00,140.000000,140.020004,139.679993,139.699997,347752
2022-06-02 14:01:00-04:00,150.279999,150.330002,150.235001,150.294998,97170
2022-06-07 14:29:00-04:00,148.860001,148.865005,148.750000,148.755005,102829
2022-06-09 13:46:00-04:00,145.669998,145.735001,145.669998,145.699997,88707
2022-06-03 10:56:00-04:00,145.570007,145.639999,145.460007,145.580994,130995
...,...,...,...,...,...
2022-06-06 13:45:00-04:00,146.565002,146.619995,146.470001,146.475098,132806
2022-06-08 12:23:00-04:00,149.000000,149.018402,148.839005,148.878998,88921
2022-06-03 15:45:00-04:00,145.184998,145.240005,145.089996,145.119705,205491


Sequential Class is a linear stack of neural network layers of arbitrary activation function

In [13]:
model = tf.keras.Sequential() 

<IPython.core.display.Javascript object>

Add two layers of sigmoid activation functions

In [14]:
model.add(Dense(16, input_dim=5, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))


In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 16)                96        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
Total params: 232
Trainable params: 232
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50)
model.evaluate(X_test, Y_test)

Epoch 1/50
58/58 [==============================] - 1s 2ms/step - loss: 2.1616 - accuracy: 0.3158
Epoch 2/50
58/58 [==============================] - 0s 2ms/step - loss: 1.4401 - accuracy: 0.5986
Epoch 3/50
58/58 [==============================] - 0s 2ms/step - loss: 1.0839 - accuracy: 0.5991
Epoch 4/50
58/58 [==============================] - 0s 2ms/step - loss: 0.8983 - accuracy: 0.5991
Epoch 5/50
58/58 [==============================] - 0s 2ms/step - loss: 0.8095 - accuracy: 0.5991
Epoch 6/50
58/58 [==============================] - 0s 2ms/step - loss: 0.7661 - accuracy: 0.5991
Epoch 7/50
58/58 [==============================] - 0s 2ms/step - loss: 0.7418 - accuracy: 0.5991
Epoch 8/50
58/58 [==============================] - 0s 2ms/step - loss: 0.7265 - accuracy: 0.5991
Epoch 9/50
58/58 [==============================] - 0s 2ms/step - loss: 0.7160 - accuracy: 0.5991
Epoch 10/50
58/58 [==============================] - 0s 2ms/step - loss: 0.7087 - accuracy: 0.5991
Epoch 11/50
58/58 [

[0.6614301204681396, 0.6314935088157654]